### Simple Gen AI APP Using LangChain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
### LangSmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [1]:
## Data Ingestion -- From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/administration-overview")
loader

In [3]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='Overview - Docs by LangChainSkip to main content🚀 Share how you\'re building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageResource HierarchyOrganizationsWorkspacesResource tagsUser Management and RBACUsersAPI keysExpiration DatesPersonal Access Tokens (PATs)Service

In [ ]:
### Load Data --> Docs --> Divide out Documents into chuncks documents --> text --> vectors --> Vector Embedding --> Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content="Overview - Docs by LangChainSkip to main content🚀 Share how you're building agents for a chance to win LangChain swag!Docs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesReleases & changelogsData managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusOn this pageResource HierarchyOrganizationsWorkspacesResource tagsUser Management and RBACUsersAPI keysExpiration DatesPersonal Access Tokens (PATs)Service 

In [6]:
### Use OpenAI Embedding technique
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [11]:
print(embeddings.model)

text-embedding-ada-002


In [ ]:
### Create Embeddings and save them into FAISS
from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)

In [8]:
vectorstoredb

In [ ]:
### Queyr From a Vector DB
query = "An organization is a logical grouping of users within LangSmith with its own"
result = vectorstoredb.similarity_search(query)
result[0]

Document(id='3ef73fb0-45ee-4de8-aa9c-e19fec30c0f4', metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='\u200bResource Hierarchy\n\u200bOrganizations\nAn organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, there is one organization per company. An organization can have multiple workspaces. For more details, see the setup guide.\nWhen you log in for the first time, a personal organization will be created for you automatically. If you’d like to collaborate with others, you can create a separate organization and invite your team members to join. There are a few important differences between your personal organization and shared organizations:\nFeaturePersonalSharedMaximum workspaces1Variable, depending on plan (see pricing pageCollaborationCannot invite usersCan invite usersBilling: paid plansDeveloper plan onlyAll other plans av

In [17]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [18]:
## Retrieval Chain, Document Chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
        Answer the following question based only on the provider context:
        <context>
            {context}
        </context>
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provider context:\n        <context>\n            {context}\n        </context>\n    '), additional_kwargs={})])
| ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<

In [21]:
from langchain_core.documents import Document
document_chain.invoke({
    "input" : "An organization is a logical grouping of users within LangSmith with its own",
    "context" : [Document(page_content="An organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, there is one organization per company.")]
})

'Based on the provided context, an organization in LangSmith is defined as a logical grouping of users within LangSmith that has its own billing configuration. Typically, each company would have one organization.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [35]:
### Input ---> Retriever ---> vectorstoreddb

vectorstoredb

In [23]:
from langchain_classic.chains import create_retrieval_chain

retriever = vectorstoredb.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [24]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x30528a450>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        Answer the following question based only on the provider context:\n        <context>\n            {context}\n        </context>\n    '), additiona

In [31]:
## Get the response from the LLM
response = retrieval_chain.invoke({"input": "An organization is a logical grouping of users within LangSmith with its own"})
response

{'input': 'An organization is a logical grouping of users within LangSmith with its own',
 'context': [Document(id='3ef73fb0-45ee-4de8-aa9c-e19fec30c0f4', metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='\u200bResource Hierarchy\n\u200bOrganizations\nAn organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, there is one organization per company. An organization can have multiple workspaces. For more details, see the setup guide.\nWhen you log in for the first time, a personal organization will be created for you automatically. If you’d like to collaborate with others, you can create a separate organization and invite your team members to join. There are a few important differences between your personal organization and shared organizations:\nFeaturePersonalSharedMaximum workspaces1Variable, depending on plan (see pricing pageC

In [32]:
print(response['answer'])

Based on the provided context, here is a summary regarding organizations and workspaces within LangSmith:

- **Organizations**: These are logical groupings of users with their own billing configurations. An organization can have multiple workspaces and is typically aligned per company. Upon logging in for the first time, a personal organization is automatically created for a user. If collaboration is needed, users can create a shared organization and invite others.

- **Features Comparison**:
  - Personal organizations have a maximum of one workspace and cannot invite other users. They are limited to the Developer plan.
  - Shared organizations can have multiple workspaces (as allowed by the specific plan and pricing), can invite users, and have access to all other plans beyond the Developer plan.

- **Workspaces**: These were formerly referred to as Tenants. They are subdivisions within an organization.

- **User Management and Roles**: 
  - Users can be part of multiple organizations

In [34]:
response['context']

[Document(id='3ef73fb0-45ee-4de8-aa9c-e19fec30c0f4', metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='\u200bResource Hierarchy\n\u200bOrganizations\nAn organization is a logical grouping of users within LangSmith with its own billing configuration. Typically, there is one organization per company. An organization can have multiple workspaces. For more details, see the setup guide.\nWhen you log in for the first time, a personal organization will be created for you automatically. If you’d like to collaborate with others, you can create a separate organization and invite your team members to join. There are a few important differences between your personal organization and shared organizations:\nFeaturePersonalSharedMaximum workspaces1Variable, depending on plan (see pricing pageCollaborationCannot invite usersCan invite usersBilling: paid plansDeveloper plan onlyAll other plans a